In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import brown, stopwords

In [3]:
# import corpus here
# nltk.download() 

In [13]:
print(brown.categories())


['adventure', 'belles_lettres', 'editorial', 'fiction', 'government', 'hobbies', 'humor', 'learned', 'lore', 'mystery', 'news', 'religion', 'reviews', 'romance', 'science_fiction']


## I chose to only use 5 categories in my work here because I did not want overload my kernel as I am on a MacBookAir and do not have unlimited space in memory. 

In [45]:
raw_adventure = brown.words(categories='adventure')
raw_lore = brown.words(categories='lore')
raw_mystery = brown.words(categories='mystery')
raw_romance = brown.words(categories='romance')
raw_science_fiction = brown.words(categories='science_fiction')

raw_adventure = str(raw_adventure)
raw_lore = str(raw_lore)
raw_mystery = str(raw_mystery)
raw_romance = str(raw_romance)
raw_science_fiction = str(raw_science_fiction)


In [68]:
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub('[\',]',' ',text)
    text = re.sub("[\[]*?![\]]", "", text)
    text = ' '.join(text.split())
    return text
    
clean_adventure = text_cleaner(raw_adventure)
clean_lore = text_cleaner(raw_lore)
clean_mystery = text_cleaner(raw_mystery)
clean_romance = text_cleaner(raw_romance)
clean_science_fiction = text_cleaner(raw_science_fiction)


In [69]:
clean_adventure

'[Dan Morgan told himself he would ...]'

In [70]:
nlp = spacy.load('en')

adventure_doc = nlp(clean_adventure)
#lore_doc = nlp(raw_lore)
#mystery_doc = nlp(raw_mystery)
#romance_doc = nlp(raw_romance)
#science_fiction_doc = nlp(raw_science_fiction)

In [71]:
adventure_doc

[Dan Morgan told himself he would ...]

In [72]:
adventure_sents = [[sent, "adventure"] for sent in adventure_doc]
#lore_sents = [[sent, "lore"] for sent in lore_doc]
#mystery_sents = [[sent, "mystery"] for sent in mystery_doc]
#romance_sents = [[sent, "romance"] for sent in romance_doc]
#science_fiction_sents = [[sent, "science_fiction"] for sent in science_fiction_doc]

sentences = pd.DataFrame(adventure_sents)

sentences.head()

,0,1
0,[,adventure
1,Dan,adventure
2,Morgan,adventure
3,told,adventure
4,himself,adventure


In [16]:
# Utility function to create a list of the 3000 most common words.
# upped word cound from 2000 to 3000 because we aren't comparing works from
# two different authors but from 5 different genres.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(3000)]
    

def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df


In [19]:
adventure_text = brown.words(categories='adventure')
lore_text = brown.words(categories='lore')
mystery_text = brown.words(categories='mystery')
romance_text = brown.words(categories='romance')
science_fiction_text = brown.words(categories='science_fiction')

common_words = set(adventure_text + lore_text + mystery_text + romance_text + science_fiction_text)

In [20]:
word_counts = bow_features(sentences, common_words)

AttributeError: 'str' object has no attribute 'is_punct'

In [ ]:
# set up pipeline pieces here
from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

steps = []


In [8]:
help(nltk.tokenize)

Help on package nltk.tokenize in nltk:

NAME
    nltk.tokenize - NLTK Tokenizer Package

DESCRIPTION
    Tokenizers divide strings into lists of substrings.  For example,
    tokenizers can be used to find the words and punctuation in a string:
    
        >>> from nltk.tokenize import word_tokenize
        >>> s = '''Good muffins cost $3.88\nin New York.  Please buy me
        ... two of them.\n\nThanks.'''
        >>> word_tokenize(s)
        ['Good', 'muffins', 'cost', '$', '3.88', 'in', 'New', 'York', '.',
        'Please', 'buy', 'me', 'two', 'of', 'them', '.', 'Thanks', '.']
    
    This particular tokenizer requires the Punkt sentence tokenization
    models to be installed. NLTK also provides a simpler,
    regular-expression based tokenizer, which splits text on whitespace
    and punctuation:
    
        >>> from nltk.tokenize import wordpunct_tokenize
        >>> wordpunct_tokenize(s)
        ['Good', 'muffins', 'cost', '$', '3', '.', '88', 'in', 'New', 'York', '.',
     

In [ ]:
# set up pipeline here
from sklearn.pipeline import Pipeline


In [ ]:
# put assesment here

In [ ]:
# 